In [1]:
import os
os.chdir('/Home/siv31/fla023/master/Hubert-Final-Project/project')

In [2]:
from src.utils.Environments import OpenAIEnv as Environment
from src.utils import DQN, Transition, ReplayMemory, read_tsv_file
from random import random

import matplotlib
import matplotlib.pyplot as plt

import torch
from math import exp
from torch.optim import AdamW
import torch.nn as nn
from itertools import count
from tqdm import tqdm

/Home/siv31/fla023/.conda/envs/HUBERT/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
grid = read_tsv_file('../resources/map_4.dat', enemies={}, start=(46, 1), end=(1, 46))
env = Environment(name="Level 5", grid=grid, project_path='Levels')
# env.plot(color_bar=False, save=True)

In [4]:
# HYPER PARAMETERS
BATCH_SIZE = 32
GAMMA = 0.999
EPS_START = 0.2
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

# Get number of actions from gym action space
n_actions = env.action_space.n

# Get the number of state observations
state = env.reset()

n_observations = len(state)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

policy_net = DQN(input_dim=n_observations, output_dim=n_actions, device=device)
target_net = DQN(input_dim=n_observations, output_dim=n_actions, device=device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)

In [5]:
steps_done = 0


def select_action(state):
    global steps_done
    sample = random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1).clamp(0, 2)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)

In [6]:
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

In [7]:
episode_durations = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())

    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated

    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [8]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    
    transitions = memory.sample(BATCH_SIZE)

    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), device=device,
                                  dtype=torch.bool)

    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])

    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0]

    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()

    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [9]:
if torch.cuda.is_available():
    num_episodes = 100_000
else:
    num_episodes = 1_000

print(f'[POLICY NET] {next(policy_net.parameters()).device}')
print(f'[TARGET NET] {next(target_net.parameters()).device}')
print(f'[NUM EPISODES] {num_episodes}')

rewards = []

for i_episode in tqdm(range(num_episodes), desc='Episode'):
    # Initialize the environment and get it's state
    state = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)

    total_reward = 0.0

    for t in count():
        action = select_action(state)
        if t % 10_000 == 0: print(t, action, env.y, env.x)
        observation, reward, terminated, truncated = env.step(action.item())
        total_reward += reward

        reward = torch.tensor([reward], device=device)
        done = terminated or truncated
        
        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)
            
        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()

        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key] * TAU + target_net_state_dict[key] * (1 - TAU)

        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            # plot_durations()
            break

    print(total_reward)
    rewards.append(total_reward)

print('Complete')

plot_durations(show_result=True)
plt.ioff()
plt.show()

[POLICY NET] cuda:0
[TARGET NET] cuda:0
[NUM EPISODES] 100000


Episode:   0%|          | 0/100000 [00:00<?, ?it/s]

0 tensor([[1]], device='cuda:0') 46 1
10000 tensor([[2]], device='cuda:0') 46 46
20000 tensor([[0]], device='cuda:0') 46 1
30000 tensor([[0]], device='cuda:0') 46 1
40000 tensor([[0]], device='cuda:0') 46 3
50000 tensor([[2]], device='cuda:0') 46 45
60000 tensor([[2]], device='cuda:0') 46 45
70000 tensor([[2]], device='cuda:0') 46 45
80000 tensor([[2]], device='cuda:0') 46 46
90000 tensor([[2]], device='cuda:0') 46 1
100000 tensor([[2]], device='cuda:0') 46 2
110000 tensor([[2]], device='cuda:0') 46 46
120000 tensor([[2]], device='cuda:0') 46 45
130000 tensor([[0]], device='cuda:0') 46 2
140000 tensor([[2]], device='cuda:0') 46 1
150000 tensor([[2]], device='cuda:0') 46 2
160000 tensor([[2]], device='cuda:0') 46 46
170000 tensor([[0]], device='cuda:0') 46 1
180000 tensor([[2]], device='cuda:0') 46 2
190000 tensor([[0]], device='cuda:0') 46 3
200000 tensor([[2]], device='cuda:0') 46 46
210000 tensor([[2]], device='cuda:0') 46 45
220000 tensor([[2]], device='cuda:0') 46 46
230000 tensor(

Episode:   0%|          | 0/100000 [34:40<?, ?it/s]


KeyboardInterrupt: 